In [ ]:
import numpy as np
import pandas as pd
import healpy as hp
from joblib import Parallel, delayed
from tqdm import tqdm
from statistics import mean, stdev
import os

# Create directory for intermediate output
file_inter = "inter_data1"
os.makedirs(file_inter, exist_ok=True)

# Parameters
nside = 64      # Healpix resolution
nf = 100       # Number of bootstrap samples
nbin = 30      # Number of radial bins
use_cores = 15

# Compute Rényi entropy for a given dataset
def compute_renyi_entropy(input_file, output_file, rmin, rmax, Nside=nside, nbin=30):
    Npix = hp.nside2npix(Nside)
    df = pd.read_csv(input_file, sep="\t", header=None)
    df.columns = ['r', 'th', 'ph']
    r1, th1, ph1 = df['r'].to_numpy(), df['th'].to_numpy(), df['ph'].to_numpy()

    dr = (rmax - rmin) / nbin
    m = np.zeros((nbin, Npix), dtype=float)
    Neff = np.zeros(nbin, dtype=int)

    # Fill Healpix map
    for rr, tt, pp in zip(r1, th1, ph1):
        for j in range(nbin):
            if rmin <= rr <= (rmin + (j+1) * dr):
                px = hp.ang2pix(Nside, tt, pp)
                m[j][px] += 1
                Neff[j] += 1

    # Normalize to probability
    p = np.zeros((nbin, Npix), dtype=float)
    for i in range(nbin):
        if Neff[i] > 0:
            p[i] = m[i] / Neff[i]

    # Compute Rényi entropies for q=1 to 5
    H = np.zeros((nbin, 5), dtype=float)
    for k in range(5):
        q = k + 1
        for i in range(nbin):
            if q == 1:
                H[i][k] = -np.sum(p[i][p[i] > 0] * np.log10(p[i][p[i] > 0]))
            else:
                h_q = np.sum(p[i] ** q)
                if h_q > 0:
                    H[i][k] = np.log10(h_q) / (1 - q)

    R = rmin + (np.arange(1, nbin + 1) * dr)
    df_out = pd.concat([pd.DataFrame(R), pd.DataFrame(H)], axis=1)
    df_out.to_csv(output_file, sep='\t', header=False, index=False)

# Helper for bootstrapping
def process_sample(l, df_base, R_min, R_max, file_inter):
    fd = df_base.sample(frac=0.8, replace=True, random_state=l)
    f_samp = f"{file_inter}/sample2_{l+1}.dat"
    f_out2 = f"{file_inter}/anis_s2_{l+1}.dat"
    fd.to_csv(f_samp, sep="\t", header=None, index=False)
    compute_renyi_entropy(f_samp, f_out2, R_min, R_max)

# Read input sample
f_in = '../Data_exc_lqg_s2/m4_sample2_ex_LQG.dat'
df = pd.read_csv(f_in, sep="\t", header=None)
df.columns = ['r', 'th', 'ph']

# Compute global r min/max
R_min = df['r'].min()
R_max = df['r'].max()

# Original entropy computation
f_out1 = f"{file_inter}/renyi_anis_s2.dat"
compute_renyi_entropy(f_in, f_out1, R_min, R_max)

# Bootstrap entropy (parallelized)
from joblib import parallel_backend

# Bootstrap entropy (parallelized with specified backend)
with parallel_backend('loky', n_jobs=use_cores):
    Parallel()(
        delayed(process_sample)(l, df, R_min, R_max, file_inter)
        for l in range(nf))
########################################################################################################

from statistics import mean, stdev
nbin = 30

# ---- Compute normalized entropy dispersion and save ----
file = f'inter_data1/renyi_anis_s2.dat'
RR = np.loadtxt(file)[:, 0]
s1 = np.loadtxt(file)[:, 1]
s2 = np.loadtxt(file)[:, 2]
s3 = np.loadtxt(file)[:, 3]
s4 = np.loadtxt(file)[:, 4]
s5 = np.loadtxt(file)[:, 5]

stab_cri = np.zeros(len(RR))
s_mean = np.zeros(len(RR))
for i in range(len(RR)):
    s_mean[i] = (s1[i] + s2[i] + s3[i] + s4[i] + s5[i]) / 5.0
    stab_cri[i] = np.sqrt((1 / 5) * ((s1[i] - s_mean[i]) ** 2 + (s2[i] - s_mean[i]) ** 2 +
                                     (s3[i] - s_mean[i]) ** 2 + (s4[i] - s_mean[i]) ** 2 +
                                     (s5[i] - s_mean[i]) ** 2))
frac_cri = stab_cri / s_mean
dict1 = {'r': RR, 'crit': frac_cri}
fd = pd.DataFrame(dict1)
file_out = f'inter_data1/sample2_crit.csv'
fd.to_csv(file_out, index=False)

# ---- Compute error on entropy criteria over bootstrap samples ----
file_cri = f'inter_data1/sample2_crit.csv'
df_cri = pd.read_csv(file_cri)
crit_samp = df_cri['crit'].to_numpy()

criteria = np.zeros((nf, nbin))
d_crit = np.zeros(nbin)

for f in range(nf):
    file = f'inter_data1/anis_s2_{f+1}.dat'
    RR = np.loadtxt(file)[:, 0]
    a1 = np.loadtxt(file)[:, 1]
    a2 = np.loadtxt(file)[:, 2]
    a3 = np.loadtxt(file)[:, 3]
    a4 = np.loadtxt(file)[:, 4]
    a5 = np.loadtxt(file)[:, 5]
    a_mean = np.zeros(len(RR))
    stab_cri = np.zeros(len(RR))

    for i in range(len(RR)):
        a_mean[i] = (a1[i] + a2[i] + a3[i] + a4[i] + a5[i]) / 5.0
        stab_cri[i] = np.sqrt((1 / 5) * ((a1[i] - a_mean[i]) ** 2 + (a2[i] - a_mean[i]) ** 2 +
                                        (a3[i] - a_mean[i]) ** 2 + (a4[i] - a_mean[i]) ** 2 +
                                        (a5[i] - a_mean[i]) ** 2))
    criteria[f] = stab_cri / a_mean

for i in range(nbin):
    list_crit = [criteria[j][i] for j in range(nf)]
    d_crit[i] = stdev(list_crit)

dict1 = {'r': RR, 'crit': crit_samp, 'sd': d_crit}
fd_out = pd.DataFrame(dict1)
f_name1 = f'inter_data1/sample_2_criteria_err.csv'
fd_out.to_csv(f_name1, index=False)

#####################################################################################################

f_out1 = 'inter_data1/renyi_anis_s2.dat'
os.remove(f_out1)
f_out2 = 'inter_data1/sample2_crit.csv'
os.remove(f_out2)
for l in range(nf):
    file_name = 'inter_data1/anis_s2' + '_' + str (l+1) + '.dat'
    os.remove(file_name)
    f_samp = 'inter_data1/sample2'+ '_' + str(l+1) + '.dat' 
    os.remove(f_samp)
